In [1]:
from gasp.cpu.arcg.spanlst.view import generate_obs_points_for_large_viewshed

LULC_RASTER  = r'C:\giswork\ext_cgi\event_0_lulc_1.tif'
DEM_RASTER   = r'C:\giswork\ext_cgi\srtm_inrst.tif'
REFCELLS_SHP = r'C:\giswork\ext_cgi\grid_10x10km.shp'
ID_CELLS     = "id_cell"
OUT_FOLDER   = r'C:\giswork\ext_cgi\view_0_1'
SNAP_RASTER  = r'C:\giswork\ext_cgi\rst_ref.tif'
EPSG         = 3763
TOLERANCE    = 500

generate_obs_points_for_large_viewshed(
    LULC_RASTER, DEM_RASTER, REFCELLS_SHP,
    ID_CELLS, OUT_FOLDER, SNAP_RASTER,
    EPSG, TOLERANCE,
    PRODUCE_DEM=None, PRODUCE_FROM_INDEX=None
)

KeyboardInterrupt: 

In [ ]:
import os
from threading              import Thread
from gasp.oss               import get_filename
from gasp.oss.ops           import create_folder
from gasp.arcg.mng.rst.proc import clip_raster_each_feat_class
from gasp.arcg.spanlst.rcls import rcls_folderRaster
from gasp.arcg.spanlst.view import generalize_obs_points_forFolder
from gasp.pnd.anls.exct     import split_shp_by_attr
from gasp.pnd.mng.ext       import buffer_extent
from gasp.arcg.spanlst.surf import viewshed
from gasp.fmshp             import shp_to_df
from gasp.pnd.mng.fld       import col_distinct

INTEREST_RASTER = r'C:\gis\EXT_CGI\validation\cls_interesse_bin\event_0_lulc_1.tif'
DEM             = r'C:\gis\EXT_CGI\base\dem\srtm\srtm_inrst.tif'
REF_CELLS       = r'C:\gis\EXT_CGI\ref_grids\grid_10x10km.shp'
ID_CELLS        = "id_cell"
OUT_FOLDER      = r'C:\gis\EXT_CGI\validation\view_0_1'
SNAP_RASTER     = r'C:\gis\EXT_CGI\validation\rst_ref.tif'
SRS_EPSG        = 3763

C_CELLS_FLD = os.path.join(OUT_FOLDER, 'cells')
RST_TMP     = os.path.join(OUT_FOLDER, 'mask_tmp')
RST_FLD     = os.path.join(OUT_FOLDER, 'mask_rst')
B_CELLS_FLD = os.path.join(OUT_FOLDER, 'bf_cells')
OBS_FLD     = os.path.join(OUT_FOLDER, 'obs_pnt')
DEM_RST     = os.path.join(OUT_FOLDER, get_filename(DEM))

THRD_MAPS = {
    "INT_RST" : {
        "CELLS_FLD" : create_folder(C_CELLS_FLD, overwrite=None) if not os.path.exists(C_CELLS_FLD) else C_CELLS_FLD,
        "RST_TMP"   : create_folder(RST_TMP, overwrite=None) if not os.path.exists(RST_TMP) else RST_TMP,
        "RST_FLD"   : create_folder(RST_FLD, overwrite=None) if not os.path.exists(RST_FLD) else RST_FLD,
        "OBS_FLD"   : create_folder(OBS_FLD, overwrite=None) if not os.path.exists(OBS_FLD) else OBS_FLD
    },
    "DEM"   : {
        "RST_FLD"   : create_folder(DEM_RST, overwrite=True) if not os.path.exists(DEM_RST) else DEM_RST,
        "CELLS_FLD" : create_folder(B_CELLS_FLD, overwrite=None) if not os.path.exists(B_CELLS_FLD) else B_CELLS_FLD,
    }
}

In [ ]:
"""
Split Cells
"""

def split_cells(f, bf, of):
    if bf == "DEM":
        _f = buffer_extent(f, SRS_EPSG, 10000, os.path.join(OUT_FOLDER, 'bf_cells.shp'))
    
    else:
        _f = f
    
    split_shp_by_attr(_f, ID_CELLS, of["CELLS_FLD"], _format='.shp')


thrds = [Thread(
    name='split-{}'.format(k), target=split_cells, args=(REF_CELLS, k, THRD_MAPS[k])
) for k in THRD_MAPS]

for t in thrds:
    t.start()

for t in thrds:
    t.join()

In [ ]:
"""
Clip INTEREST RASTER for all cells in CELLS_FLD and BF_CELLS_FLD
"""

for key, val in THRD_MAPS.items():
    inrst = DEM if key == "DEM" else INTEREST_RASTER

    clip_raster_each_feat_class(
        inrst, val["CELLS_FLD"],
        val["RST_FLD"] if key == "DEM" else val["RST_TMP"],
        snap=SNAP_RASTER, clipGeometry=True
    )

In [ ]:
"""
Reclassify Rasters of interest:

- For each cell, create a Raster with presence of class and other with ausence of class
"""
    
rcls_folderRaster(THRD_MAPS["INT_RST"]["RST_TMP"], {0 : 'NoData', 1 : 1}, THRD_MAPS["INT_RST"]["RST_FLD"])

In [ ]:
"""
Create Observer points for each raster
"""

generalize_obs_points_forFolder(
    THRD_MAPS["INT_RST"]["RST_FLD"], 500, THRD_MAPS["INT_RST"]["OBS_FLD"]
)

In [ ]:
cellsDf = shp_to_df(REF_CELLS)

LST_CELLS = col_distinct(cellsDf, ID_CELLS)

In [ ]:
import arcpy
arcpy.CheckOutExtension('Spatial')

In [ ]:
OBS_REF_NAME = get_filename(REF_CELLS)

for cell_id in LST_CELLS[:40]:
    OUT_RST = os.path.join(OUT_FOLDER, "vis_{}.tif".format(str(cell_id)))
    
    if os.path.exists(OUT_RST):
        continue
    
    DEM_RST = os.path.join(THRD_MAPS["DEM"]["RST_FLD"], "bf_cells_{}.tif".format(str(cell_id)))
    
    OBS_SHP = os.path.join(THRD_MAPS["INT_RST"]["OBS_FLD"], "{}_{}.shp".format(OBS_REF_NAME, str(int(cell_id))))
    if not os.path.exists(OBS_SHP):
        continue
    
    viewshed(
        DEM_RST, OBS_SHP, OUT_RST, snapRaster=DEM_RST, extRaster=DEM_RST
    )

In [ ]:
print LST_CELLS[:5]